In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
import scipy
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 20.0
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
import pickle

In [3]:
%load_ext autoreload
%autoreload 2


In [66]:
import graph_learning_utils as gl

# Summary of this notebook

* Implement faster, sparse-matrix based ways of generate Boolean sampling data

* Implement a simple occlusion method that zeroes out rows/columns of a matrix

* Test the solver on occluded versions of human PPI data. Unfortunately, it is simply too slow. 

We need to either run sparsity based methods, or implement on GPUs. 

# Occlusion model 

We are given a ground truth $A^s$ through an unbiased sampling procedure from the base PPI. 

We generate some number of P_i by knocking certain edges from P, then sampling from than biased version to get P_i. 

Run the algorithm, then ask: 

* How many edges can we recover?

Can use the usual binary classification metrics, e.g. precision, recall, F1, etc.

## Import data

In [5]:
sparse_matrix = scipy.sparse.load_npz('data/adj_matrix_sparse_restricted_9606.npz')

In [6]:
sparse_matrix

<11916x11916 sparse matrix of type '<class 'numpy.float64'>'
	with 5963604 stored elements in Compressed Sparse Row format>

In [8]:
max_degree =  np.max(sparse_matrix)
sparse_matrix /= max_degree

### faster bernoulli sampling

In [54]:
def gen_occluded_p(sparse_M, frac_to_occlude = 0.01): 
    n = sparse_M.shape[0]
    num_to_occlude = int(frac_to_occlude * n)
    occluded_indices = np.random.choice(n, size=num_to_occlude, replace=False)
    return zero_rows_cols(sparse_M, occluded_indices)

def zero_rows_cols(M, row_indices):
    '''
    Zeroes out the rows/cols in row_indices. 
    M is a sparse matrix type. 
    '''
    diag = scipy.sparse.eye(M.shape[0]).tolil()
    for r in row_indices:
        diag[r, r] = 0
    return diag.dot(M).dot(diag)

def bin_samples_rand2(n, mu, seed=0):
    '''
    mu: List of floats in [0,1] describing Bernoulli mean probabilities. 
    n: number of samples per entries of mu. 
    '''
    rng = np.random.default_rng(seed)
    return (rng.random(size=(len(mu), n)) < mu[:, None]).astype(np.uint8)

def gen_sparse_sample_boolean_mat(sparse_mat): 
    nonzero_float_entries = scipy.sparse.triu(sparse_mat, k=1).data
    sparse_tri = scipy.sparse.csr_matrix(scipy.sparse.triu(sparse_mat, k = 1))
    sample_bool = bin_samples_rand2(1, nonzero_float_entries).squeeze(-1)
    sparse_tri.data = sample_bool
    symm_sparse = sparse_tri + sparse_tri.T
    return symm_sparse

In [52]:
m = 5 
occluded_ground_truths = [gen_occluded_p(sparse_matrix) for _ in range(m)]

In [ ]:
# nonzero_float_entries = scipy.sparse.triu(sparse_matrix, k=1).data
# sparse_tri = scipy.sparse.csr_matrix(scipy.sparse.triu(sparse_matrix, k = 1))
# sample_bool = bin_samples_rand2(1, nonzero_float_entries).squeeze(-1)
# sparse_tri.data = sample_bool
# symm_sparse = sparse_tri + sparse_tri.T

In [55]:
validation_mat = gen_sparse_sample_boolean_mat(sparse_matrix)
sample_mats = [gen_sparse_sample_boolean_mat(occluded_ground_truth) for occluded_ground_truth in occluded_ground_truths]

## rewrite algo for sparse matrices

In [59]:
def objective_with_params_sparse(eta_arr, validation_mat, sample_mats, delta, num_eigs_included=None, verbose=False):
    # if verbose:
    #     print(eta_arr)
    if num_eigs_included is None:
        num_eigs_included = validation_mat.shape[0]
    P_hat = gl.matrix_lin_combo_pos_sign(eta_arr, sample_mats, sparse=True)

    # singular values, decreasing order
    diff_sing_values = scipy.sparse.linalg.svds(validation_mat - P_hat, 
                                                k=num_eigs_included, return_singular_vectors=False)

    # scipy.linalg.svdvals(validation_mat - P_hat)

    def ob_fn(k): 
        return sum(diff_sing_values[:k]) - k * delta

    all_obj_values = [ob_fn(k) for k in range(1, num_eigs_included + 1)]
    max_obj_index = np.argmax(all_obj_values)

    return all_obj_values[max_obj_index]

In [60]:
eta_init = gl.generate_random_eta(len(sample_mats))

In [63]:
delta = 0.0
objective = lambda eta_arr: objective_with_params_sparse(eta_arr, validation_mat, sample_mats, delta, verbose=False)

result = scipy.optimize.minimize(
    objective,
    eta_init,
    method='SLSQP',
    jac=None,
    bounds=[(0, 1) for _ in range(len(sample_mats))],
    options={
        'maxiter': 10000,
        'disp': False
    }
)

AttributeError: 'list' object has no attribute '_swap'

## run basic algo, numpy version

In [56]:
eta_init = gl.generate_random_eta(m)

In [64]:
sample_mats_np = [sm.toarray() for sm in sample_mats]

In [65]:
validation_mat_np = validation_mat.toarray()

In [73]:
# result = gl.run_scipy_minimize(sample_mats_np, validation_mat_np, 
#                                delta=0.01, eta_init=None, verbose=True)

In [79]:
# def track(f, out_values=None, out_args=None, out_kwargs=None):
#     def wrapped(*args, **kwargs):
#         if out_args is not None: out_args.append(args)
#         if out_kwargs is not None: out_kwargs.append(kwargs)
#         val = f(*args, **kwargs)
#         if out_values is not None: out_values.append(val)
#         return val
#     return wrapped

In [80]:
# vars, vals = [], []

In [81]:
def callbackF(eta_current):
    print(eta_current)

In [82]:
delta = 0.001
verbose=True

In [84]:
# objective = lambda eta_arr: gl.objective_with_params(eta_arr, validation_mat_np, sample_mats_np, delta, verbose=False)
# print('begin iter')
# # constraints={'type': 'eq', 'fun': simplex_constraint},
# result = scipy.optimize.minimize(
#     objective,
#     gl.generate_random_eta(len(sample_mats)),
#     callback=callbackF,
#     method='SLSQP',
#     jac=None,
#     bounds=[(0, 1) for _ in range(len(sample_mats))],
#     options={
#         'maxiter': 10000,
#         'disp': verbose
#     }
# )

Result: In 4.5 minutes (on the vastness machine) we can't even perform a single iteration of the solver. 

We will need some tricks to speed this up - in particular, we need to either: 

* Do a GPU based solver 

* Use the sparse matrix structure of our data. Our average edge density is something like 4%, so this should be quite doable.

# Stochastic Blockmodel

In [98]:
k = 2 
n = 200
comm_size = int(n / k)
p_in = 0.5
p_out = 0.1
probs_mat = p_out * np.ones((2, 2)) + (p_in - p_out) * np.diag(np.ones(2))

In [99]:
probs_mat

array([[0.5, 0.1],
       [0.1, 0.5]])

In [100]:
G_sbm = nx.stochastic_block_model(sizes=[comm_size for _ in range(k)], 
                          p=probs_mat)